# Prosper Loan Data Analysis

# Deepak Sithu

## Table of Contents

* <a href="#overview">Overview</a>
* <a href="#prelim">Preliminary Wrangling</a>
* <a href="#initial">Initial Exploration</a>
* <a href="#univar">Univariate Exploration</a>
* <a href="#bivar">Bivariate Exploration</a>
* <a href="#multivar">Multivariate Exploration</a>

In [13]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

<a id="overview"></a>
## Overview

> This project is an examination of loan data from Prosper. The data set contains information for 

> From the guide: This data set contains 113,937 loans with 81 variables on each loan, including loan amount, borrower rate (or interest rate), current loan status, borrower income, and many others. I'm supposed to focus on about 10-15 of the variables. Example questions are, what factors affect a loan's outcome status? What affects the borrower's APR or interest rate? Are there differences between loans depending on how large the original loan amount was?

In [28]:
# read in dataset
prosper = pd.read_csv('prosperLoanData.csv')

<a id="initial"></a>
## Initial Exploration

In [54]:
# get shape of dataframe and information for columns
print(prosper.shape)
prosper.info()

(113937, 81)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 81 columns):
ListingKey                             113937 non-null object
ListingNumber                          113937 non-null int64
ListingCreationDate                    113937 non-null object
CreditGrade                            28953 non-null object
Term                                   113937 non-null int64
LoanStatus                             113937 non-null object
ClosedDate                             55089 non-null object
BorrowerAPR                            113912 non-null float64
BorrowerRate                           113937 non-null float64
LenderYield                            113937 non-null float64
EstimatedEffectiveYield                84853 non-null float64
EstimatedLoss                          84853 non-null float64
EstimatedReturn                        84853 non-null float64
ProsperRating (numeric)                84853 non-null float64
ProsperRa

In [55]:
# descriptions of numeric columns
prosper.describe()

,ListingNumber,Term,BorrowerAPR,BorrowerRate,LenderYield,EstimatedEffectiveYield,EstimatedLoss,EstimatedReturn,ProsperRating (numeric),ProsperScore,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
count,1.139370e+05,113937.000000,113912.000000,113937.000000,113937.000000,84853.000000,84853.000000,84853.000000,84853.000000,84853.000000,...,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000,113937.000000
mean,6.278857e+05,40.830248,0.218828,0.192764,0.182701,0.168661,0.080306,0.096068,4.072243,5.950067,...,-54.725641,-14.242698,700.446342,681.420499,25.142686,0.998584,0.048027,0.023460,16.550751,80.475228
std,3.280762e+05,10.436212,0.080364,0.074818,0.074516,0.068467,0.046764,0.030403,1.673227,2.376501,...,60.675425,109.232758,2388.513831,2357.167068,275.657937,0.017919,0.332353,0.232412,294.545422,103.239020
min,4.000000e+00,12.000000,0.006530,0.000000,-0.010000,-0.182700,0.004900,-0.182700,1.000000,1.000000,...,-664.870000,-9274.750000,-94.200000,-954.550000,0.000000,0.700000,0.000000,0.000000,0.000000,1.000000
25%,4.009190e+05,36.000000,0.156290,0.134000,0.124200,0.115670,0.042400,0.074080,3.000000,4.000000,...,-73.180000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000
50%,6.005540e+05,36.000000,0.209760,0.184000,0.173000,0.161500,0.072400,0.091700,4.000000,6.000000,...,-34.440000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,44.000000
75%,8.926340e+05,36.000000,0.283810,0.250000,0.240000,0.224300,0.112000,0.116600,5.000000,8.000000,...,-13.920000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,115.000000
max,1.255725e+06,60.000000,0.512290,0.497500,0.492500,0.319900,0.366000,0.283700,7.000000,11.000000,...,32.060000,0.000000,25000.000000,25000.000000,21117.900000,1.012500,39.000000,33.000000,25000.000000,1189.000000


In [56]:
# head of dataframe
prosper.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


### What is the structure of the dataset?

> The dataset is a series of loans with a large amount of associated data for each. One interesting aspect is that the credit grading system changed in July 2009, so credit ratings are split between the columns CreditGrade and ProsperRating (Alpha). It's clear from the initial exploration into the dataset that, both for ease of use and for the scope of this project it is best to eliminate several of the columns which are not going to be relavent. 

### What are the main features of interest in the dataset?

> The main features of interest are the credit grades, which are split between CreditGrade and ProsperRating (Alpha), the LoanStatus, the BorrowerAPR, and the BorrowerRate. It will be interesting to see what, if any of the other variables in the dataset are correlated with differences in the previously mentioned features. Many of these are features which are  determined by the loan company, and therefore out of the control of the borrower, so seeing the relationship between these and variables which are more borrower-based will provide an insight into what goes into determining the specifics of a loan.

### What features in the dataset do you think will help support your investigation into the features of interest?

> The features which will be explored to see if they support an investigation into the features of interest are Term, ListingCategory, BorrowerState, EmploymentStatus, IsBorrowerHomeowner, OpenCreditLines, TotalCreditLinespast7years, DebtToIncomeRatio, and IncomeRange.

<a id="prelim"></a>
## Preliminary Wrangling

In [66]:
# create copy of dataframe with only columns of interest and confirm
prosper_clean = prosper[['ListingKey', 'CreditGrade', 'Term', 'LoanStatus', 'BorrowerAPR', 'BorrowerRate', 'ProsperRating (Alpha)', 'ListingCategory (numeric)', 'BorrowerState', 'EmploymentStatus', 'IsBorrowerHomeowner', 'OpenCreditLines', 'TotalCreditLinespast7years', 'DebtToIncomeRatio', 'IncomeRange']].copy()
print(prosper_clean.shape)
prosper_clean.head()

(113937, 15)


,ListingKey,CreditGrade,Term,LoanStatus,BorrowerAPR,BorrowerRate,ProsperRating (Alpha),ListingCategory (numeric),BorrowerState,EmploymentStatus,IsBorrowerHomeowner,OpenCreditLines,TotalCreditLinespast7years,DebtToIncomeRatio,IncomeRange
0,1021339766868145413AB3B,C,36,Completed,0.16516,0.1580,NaN,0,CO,Self-employed,True,4.0,12.0,0.17,"$25,000-49,999"
1,10273602499503308B223C1,NaN,36,Current,0.12016,0.0920,A,2,CO,Employed,False,14.0,29.0,0.18,"$50,000-74,999"
2,0EE9337825851032864889A,HR,36,Completed,0.28269,0.2750,NaN,0,GA,Not available,False,NaN,3.0,0.06,Not displayed
3,0EF5356002482715299901A,NaN,36,Current,0.12528,0.0974,A,16,GA,Employed,True,5.0,29.0,0.15,"$25,000-49,999"
4,0F023589499656230C5E3E2,NaN,36,Current,0.24614,0.2085,D,2,MN,Employed,True,19.0,49.0,0.26,"$100,000+"


Remaining variables and meanings:
a1. ListingKey - ID, not for analysis - full
2. CreditGrade - categorical (8) - 28953
3. Term - categorical (3 (12, 36, 60 aka 1, 3, 5 year loans)) - full
4. LoanStatus - categorical (12) - full
5. BorrowerAPR - numeric - nearly full
6. BorrowerRate - numeric - full
7. ProsperRating (Alpha) - categorical (7) (CreditGrade for loans after July 2009) - 84853
8. ListingCategory (numeric) - categorical but need to translate it from numeric (20 categories) - full
9. BorrowerState - categorical (51) - some empty but mostly full
10. EmploymentStatus - categorical (8) - some emp but most full
11. IsBorrowerHomeowner - boolean - full
12. OpenCreditLines - numeric - some emp but mos full
13. TotalCreditLinespast7years - numeric - sebmf
14. DebtToIncomeRatio - numeric - sebmf
15. IncomeRange - categorical (8) - full

In [73]:
# rename ProsperRating (Alpha) and ListingCategory (numeric) for ease of use and confirm
prosper_clean = prosper_clean.rename(columns={'ProsperRating (Alpha)' : 'ProsperRating', 'ListingCategory (numeric)' : 'ListingCategory'});
prosper_clean.head()

,ListingKey,CreditGrade,Term,LoanStatus,BorrowerAPR,BorrowerRate,ProsperRating,ListingCategory,BorrowerState,EmploymentStatus,IsBorrowerHomeowner,OpenCreditLines,TotalCreditLinespast7years,DebtToIncomeRatio,IncomeRange
0,1021339766868145413AB3B,C,36,Completed,0.16516,0.1580,NaN,0,CO,Self-employed,True,4.0,12.0,0.17,"$25,000-49,999"
1,10273602499503308B223C1,NaN,36,Current,0.12016,0.0920,A,2,CO,Employed,False,14.0,29.0,0.18,"$50,000-74,999"
2,0EE9337825851032864889A,HR,36,Completed,0.28269,0.2750,NaN,0,GA,Not available,False,NaN,3.0,0.06,Not displayed
3,0EF5356002482715299901A,NaN,36,Current,0.12528,0.0974,A,16,GA,Employed,True,5.0,29.0,0.15,"$25,000-49,999"
4,0F023589499656230C5E3E2,NaN,36,Current,0.24614,0.2085,D,2,MN,Employed,True,19.0,49.0,0.26,"$100,000+"


In [94]:
# confirm whether there are any overlaps between CreditGrade and ProsperRating
# create two lists of the ListingKeys for non-null values for CreditGrade and ProsperRating
credit_grade_keys = prosper_clean.ListingKey[prosper_clean['CreditGrade'].notnull()].tolist();
prosper_rating_keys = prosper_clean.ListingKey[prosper_clean['ProsperRating'].notnull()].tolist();

In [95]:
# use python in and loop to check if any value in credit_grade_list is in prosper_rating_list
double_rating_keys = []
for i in range(len(credit_grade_keys)):
    if credit_grade_keys[i] in prosper_rating_keys:
        double_rating_keys.append(credit_grade_keys[i])
        print(credit_grade_keys[i])

In [96]:
# confirm double_rating_keys is empty
double_rating_keys

[]

In [97]:
# create CreditRating column which is combination of CreditGrade and ProsperRating
prosper_clean['CreditRating'] = np.nan
prosper_clean.head()

,ListingKey,CreditGrade,Term,LoanStatus,BorrowerAPR,BorrowerRate,ProsperRating,ListingCategory,BorrowerState,EmploymentStatus,IsBorrowerHomeowner,OpenCreditLines,TotalCreditLinespast7years,DebtToIncomeRatio,IncomeRange,CreditRating
0,1021339766868145413AB3B,C,36,Completed,0.16516,0.1580,NaN,0,CO,Self-employed,True,4.0,12.0,0.17,"$25,000-49,999",NaN
1,10273602499503308B223C1,NaN,36,Current,0.12016,0.0920,A,2,CO,Employed,False,14.0,29.0,0.18,"$50,000-74,999",NaN
2,0EE9337825851032864889A,HR,36,Completed,0.28269,0.2750,NaN,0,GA,Not available,False,NaN,3.0,0.06,Not displayed,NaN
3,0EF5356002482715299901A,NaN,36,Current,0.12528,0.0974,A,16,GA,Employed,True,5.0,29.0,0.15,"$25,000-49,999",NaN
4,0F023589499656230C5E3E2,NaN,36,Current,0.24614,0.2085,D,2,MN,Employed,True,19.0,49.0,0.26,"$100,000+",NaN


In [113]:
# loop through credit_grade_keys and add the CreditGrade value
# for that key to the CreditRating column
pd.options.mode.chained_assignment = None 
for i in range(len(credit_grade_keys)):
    prosper_clean[prosper_clean['ListingKey'] == credit_grade_keys[i]]['CreditRating'][0] = prosper_clean[prosper_clean['ListingKey'] == credit_grade_keys[i]]['CreditGrade'][0]
    print(prosper_clean[prosper_clean['ListingKey'] == credit_grade_keys[i]]['CreditRating'])
    break

0   NaN
Name: CreditRating, dtype: float64


In [115]:
prosper_clean[prosper_clean['ListingKey'] == credit_grade_keys[0]]['CreditRating'][0]

nan

In [116]:
prosper_clean[prosper_clean['ListingKey'] == credit_grade_keys[0]]['CreditGrade'][0]

'C'

In [117]:
# okay so i need to use pandas at function but first need to get indexes!
prosper_clean.at[#index i need, 'column']

ValueError: At based indexing on an integer index can only have integer indexers

<a id="univar"></a>
## Univariate Exploration

In [ ]:
# Here explore distributions of individual variables,
# if you see unusual points or outliers, take a deeper look to clean things up 
# and prepare yourself to look at relationships between variables

In [ ]:
# Make sure that, after every plot or related series of plots,
#you include a markdown cell with comments about what you observed and what you plan on investigating next!!!

### Discuss the distributions of your variables of interest. Were there any unusual points? Did you need to perform any transformations?

> asdf

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> asdf

<a id="bivar"></a>
## Bivariate Exploration

In [ ]:
# In this section, investigate relationships between pairs of variables in your data
# Make sure the variables that you cover here have been introduced in some fashion in the previous section (univariate)

### Talk about some of the relationships you observed in this part of the investigation. How did the features of interest vary with other features in the dataset?

> asdf

### Did you observe any interesting relationships between the other features (not the main features of interest)?

> asdf

<a id="multivar"></a>
## Multivariate Exploration

In [ ]:
# create plots of three or more variables to investigate your data further.
# make sure your investigations are justified, and follow from your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your features of interest?

> asdf

### Were there any interesting or surprising interactions between features?

> asdf

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report! XXXXXXX